In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [2]:
def initialize_surface_code(distance):
    """
    Initialize all data qubits to |0>.
    
    Parameters:
    - distance (int): The code distance.
    
    Returns:
    - np.ndarray: A 2D array representing the data qubits, initialized to 0.
    """
    
    return np.zeros((distance, distance), dtype=int)  # return grid of (distance x distance) qubits

def generate_stabilizers(distance):
    """
    Generate stabilizers for the surface code of given distance.
    
    Parameters:
    - distance (int): The code distance.
    
    """
    stabilizers = []
    
    for row in range(distance - 1):  # Loop through rows of plaquettes
        for col in range(distance - 1):  # Loop through columns of plaquettes
            # Define a stabilizer for the current plaquette
            stabilizer = [
                (row, col),     # Top-left qubit
                (row, col + 1), # Top-right qubit
                (row + 1, col), # Bottom-left qubit
                (row + 1, col + 1)  # Bottom-right qubit
            ]
            stabilizers.append(stabilizer)
    
    return stabilizers  # A list of stabilizers, each defined by the indices of qubits it monitors

In [3]:
# Introduce random bit-flip errors on the data qubits
def introduce_errors(data_qubits, error_probability=0.1):
    for row in range(data_qubits.shape[0]):
        for col in range(data_qubits.shape[1]):
            if random.random() < error_probability:
                data_qubits[row, col] ^= 1  # Flip the qubit (0 -> 1 or 1 -> 0)
    return data_qubits

def measure_stabilizers(data_qubits, stabilizers):
    # len(stabilizers) x4 matrix to store detailed parity information
    syndromes = np.zeros((len(stabilizers), 4), dtype=int)
    for i, stabilizer in enumerate(stabilizers):
        for j, (row, col) in enumerate(stabilizer):
            syndromes[i, j] = data_qubits[row, col]  # Individual qubit parity based on stabilizer measurement 
    return syndromes


def correct_errors(data_qubits, syndromes, stabilizers):
    # Track error candidates based on overlapping stabilizers
    error_candidates = {}

    for i, syndrome in enumerate(syndromes):
        for j, parity in enumerate(syndrome):
            if parity == 1:  # Error detected for this qubit
                row, col = stabilizers[i][j]
                error_candidates[(row, col)] = error_candidates.get((row, col), 0) + 1
                #print(error_candidates) # To know which stabilizer returned parity = 1 and the flagged qubit

    # Correct qubits with the highest overlap
    for (row, col), count in error_candidates.items():
        if count > 0:  # Threshold for correction (adjust as needed)
            data_qubits[row, col] ^= 1  # Flip the qubit

    return data_qubits

In [4]:
# Main simulation loop
def simulate_surface_code(distance):
    
    if distance > 10:
        raise ValueError("Distance exceeds the maximum allowed value. Please choose a smaller distance.")
    
    # Step 1: Initialize the surface code
    data_qubits = initialize_surface_code(distance)
    print("Initial data qubits:")
    print(data_qubits)

    # Step 2: Introduce errors
    data_qubits = introduce_errors(data_qubits, error_probability=0.2)
    print("Data qubits after introducing errors:")
    print(data_qubits)

    stabilizers = generate_stabilizers(distance)
    
    # Step 3: Measure stabilizers to detect errors
    syndromes = measure_stabilizers(data_qubits, stabilizers)
    print("Syndromes (stabilizer measurements):")
    print(syndromes)

    # Step 4: Correct errors based on syndromes
    corrected_qubits = correct_errors(data_qubits, syndromes, stabilizers)
    print("Data qubits after error correction:")
    print(corrected_qubits)

    # Verify if the logical state is restored
    print("Error correction successful:", np.array_equal(corrected_qubits, initialize_surface_code(distance)))

In [ ]:
distance = 3

# Run the simulation
if __name__ == "__main__":
    simulate_surface_code(distance)


Initial data qubits:
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
Data qubits after introducing errors:
[[1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 0 0 0 0]]
Syndromes (stabilizer measurements):
[[1 0 1 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [1 0 0 1]
 [0 0 1 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 1 0 1]
 [1 0 1 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 1 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
Data qubits after error correction:
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
Error correction successful: True
